In [15]:
import os
import string
import unicodedata
from typing import Any, Dict, Iterator, List, Tuple, Union
import random
import pandas as pd

import datasets
import numpy as np
from pytorch_lightning import LightningModule, Trainer, seed_everything
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.optim import lr_scheduler
from transformers import BertJapaneseTokenizer,ElectraTokenizer,T5Tokenizer
import transformers 
from transformers import RobertaTokenizer, RobertaModel
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import callbacks
from pytorch_lightning import loggers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AdamW
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [16]:
args = {
    'random_seed': 42,  # Random Seed
    # Transformers PLM name.
    'pretrained_model': 'cl-tohoku/bert-base-japanese-whole-word-masking',
    # Optional, Transformers Tokenizer name. Overrides `pretrained_model`
    'pretrained_tokenizer': 'cl-tohoku/bert-base-japanese-whole-word-masking',
    'norm_form': 'NFKC',
    'batch_size': 8,  # <=32 for TPUv2-8
    'lr': 2e-5,  # Learning Rate
    'max_length': 384,  # Max Length input size
    'doc_stride': 128,  # The interval of the context when splitting is needed
    'epochs': 3,  # Max Epochs
    'dataset': 'SkelterLabsInc/JaQuAD',
    'huggingface_auth_token': None,
    'test_mode': False,  # Test Mode enables `fast_dev_run`
    'optimizer': 'AdamW',
    'weight_decay': 0.01,  # Weight decaying parameter for AdamW
    'lr_scheduler': 'warmup_lin',
    'warmup_ratio': 0.1,
    'fp16': False,  # Enable train on FP16 (if GPU)
    'tpu_cores': 8,  # Enable TPU with 1 core or 8 cores
    'cpu_workers': os.cpu_count(), #questionとcontextを合わせた最大語数。今回のデータセットは1300は超えない
    'note':"リクルートベースライン",
}

args

#seed値を固定
def set_seed(seed =42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
set_seed(seed=args["random_seed"])

In [17]:
datasetdict = datasets.load_dataset(
    args['dataset'], use_auth_token=args['huggingface_auth_token'])
datasetdict = datasetdict.flatten()\
            .rename_column('answers.text', 'answer')\
            .rename_column('answers.answer_start', 'answer_start')\
            .rename_column('answers.answer_type', 'answer_type')
train =pd.DataFrame(datasetdict['train'][:].values(), index=datasetdict['train'][:].keys()).T
train.head()

Using custom data configuration default
Reusing dataset ja_qu_ad (/home/s16991/.cache/huggingface/datasets/SkelterLabsInc___ja_qu_ad/default/0.1.0/5847b2e2ab5e02de284395bb15f87f13eae8f6f6ff1f01e4ee9c5c0dcf8ef8eb)


  0%|          | 0/2 [00:00<?, ?it/s]

,id,title,context,question,question_type,answer,answer_start,answer_type
0,tr-000-00-000,手塚治虫,手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989...,戦後日本のストーリー漫画の第一人者で、医学博士の一面もある漫画家は誰?,Multiple sentence reasoning,[手塚治虫],[0],[Person]
1,tr-000-00-001,手塚治虫,手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989...,手塚治虫の出身地はどこになりますか?,Syntactic variation,[兵庫県宝塚市],[130],[Location]
2,tr-000-01-000,手塚治虫,大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『少...,手塚治虫の漫画家としてのデビュー作は何かな?,Lexical variation (synonymy),[『マアチャンの日記帳』],[32],[Object]
3,tr-000-01-001,手塚治虫,大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『少...,1947年に大阪に赤本ブームが起こったのはなぜ?,Syntactic variation,[『新寶島』がベストセラーとなり],[90],[Cause]
4,tr-000-01-002,手塚治虫,大阪帝国大学附属医学専門部在学中の1946年1月1日に4コマ漫画『マアチャンの日記帳』(『少...,日本で一番初めに30分枠のテレビアニメとなった作品名は何?,Lexical variation (synonymy),[『鉄腕アトム』],[213],[Object]


In [6]:
sample_context= [train["context"][0],train["context"][100],train["context"][200]]
sample_context= [train["context"][0]]
sample_context

['手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989年(平成元年)2月9日)は、日本の漫画家、アニメーター、アニメ監督である。\n戦後日本においてストーリー漫画の第一人者として、漫画表現の開拓者的な存在として活躍した。\n\n兵庫県宝塚市出身(出生は大阪府豊能郡豊中町、現在の豊中市)同市名誉市民である。\n大阪帝国大学附属医学専門部を卒業。\n医師免許取得のち医学博士(奈良県立医科大学・1961年)。']

In [7]:
bert_tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
BertJapanese_tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
electra_tokenizer = AutoTokenizer.from_pretrained("Cinnamon/electra-small-japanese-generator")
electra_tokenizer = AutoTokenizer.from_pretrained("Cinnamon/electra-small-japanese-generator")
electra_tokenizer = AutoTokenizer.from_pretrained("Cinnamon/electra-small-japanese-generator")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from you

In [9]:
def tokenizer(pretrained_tokenizer = "cl-tohoku/bert-base-japanese-whole-word-masking",Tokenizer= AutoTokenizer,contexts= sample_context):
    Tokenizer = Tokenizer.from_pretrained(pretrained_tokenizer)
    for i in range(len(contexts)):
        print("オリジナル")
        print(contexts[i])
        print(pretrained_tokenizer)
        encoding = Tokenizer.encode(contexts[i])
        tokens = Tokenizer.convert_ids_to_tokens(encoding)
        print(tokens)
        print("------------------------------------------------------------------------")
    

In [18]:
tokenizer(pretrained_tokenizer = "cl-tohoku/bert-base-japanese-whole-word-masking",Tokenizer= AutoTokenizer)
tokenizer(pretrained_tokenizer = "rinna/japanese-roberta-base",Tokenizer= AutoTokenizer)
# tokenizer(pretrained_tokenizer = "rinna/japanese-roberta-base",Tokenizer= BertJapaneseTokenizer)
tokenizer(pretrained_tokenizer = "rinna/japanese-roberta-base",Tokenizer= T5Tokenizer)
# tokenizer(pretrained_tokenizer = "cl-tohoku/bert-base-japanese-whole-word-masking",Tokenizer= AutoTokenizer)

オリジナル
手塚治虫(てづかおさむ、本名:手塚治(読み同じ)、1928年(昭和3年)11月3日-1989年(平成元年)2月9日)は、日本の漫画家、アニメーター、アニメ監督である。
戦後日本においてストーリー漫画の第一人者として、漫画表現の開拓者的な存在として活躍した。

兵庫県宝塚市出身(出生は大阪府豊能郡豊中町、現在の豊中市)同市名誉市民である。
大阪帝国大学附属医学専門部を卒業。
医師免許取得のち医学博士(奈良県立医科大学・1961年)。
cl-tohoku/bert-base-japanese-whole-word-masking
['[CLS]', '手塚', '治虫', '(', 'て', 'づか', 'お', 'さ', '##む', '、', '本名', ':', '手塚', '治', '(', '読み', '同じ', ')、', '1928', '年', '(', '昭和', '3', '年', ')', '11', '月', '3', '日', '-', '1989', '年', '(', '平成', '元年', ')', '2', '月', '9', '日', ')', 'は', '、', '日本', 'の', '漫画', '家', '、', 'アニメーター', '、', 'アニメ', '監督', 'で', 'ある', '。', '戦後', '日本', 'において', 'ストーリー', '漫画', 'の', '第一人者', 'として', '、', '漫画', '表現', 'の', '開拓', '者', '的', 'な', '存在', 'として', '活躍', 'し', 'た', '。', '兵庫', '県', '宝塚', '市', '出身', '(', '出生', 'は', '大阪', '府', '豊', '##能', '郡', '豊中', '町', '、', '現在', 'の', '豊中', '市', ')', '同市', '名誉', '市民', 'で', 'ある', '。', '大阪', '帝国', '大学', '附属', '医学', '専門', '部', 'を', '卒業', '。', '医師', '免許', '取得', 'のち', '医学', '博士', '(', '奈良', '##県立', '##医科大学', '・', '1961', '